In [1]:
import csv
import statistics
import numpy as np
from dataclasses import dataclass
from typing import Callable, List
from sklearn.linear_model import SGDRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from model.Model import Model, ZeroModel, Baseline, Ensemble
from model.TagModel import TagModel
from model.TagwiseModel import TagwiseModel
from utils.PlotBuilder import PlotBuilder
from utils.Post import Post, to_first_letter, to_same_tag, to_numbered_group
from utils.Window import Scorer, Window
from utils.WindowsManager import WindowsManager

class ModelTest:
    def __init__(self, name: str, factory: Callable[[], Model]):
        self.name = name
        self.factory = factory
        self.score = []

In [2]:
file = "./sentiment.csv"
with open(file, newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    raw_posts = [(row["Text"], row["Time"]) for row in reader
             # наиболее плотный непрерывный участок с постами:
             if row["Time"][4:10] == "Jun 07" or row["Time"][4:10] == "Jun 06"]
print("Posts total", len(raw_posts))

Posts total 208026


In [3]:
@dataclass
class Metrics:
    med_rmse: float
    avg_rmse: float

# Класс для подсчета метрик
class MetricsCalculator:
    def __init__(self, windows: List[Window]):
        self.windows = windows
        self.scorer = Scorer()

    def metrics(self, model: Model, log=False) -> Metrics:
        self.scorer.reset()

        rmse = []
        ind = 1
        total = len(self.windows)
        # Итерируемся по парам последовательных окон
        for first, second in zip(self.windows, self.windows[1:]):
            if log:
                print(ind, total)
            ind += 1
            # Обновляем scorer тегами из первого окна, чтобы те учитывались при подсчете ошибки
            self.scorer.update([tag for post in first.posts for tag in post.tags])
            # Заполняем список RMSE для пар окон
            # scorer считает RMSE на всех встретившихся ранее тегах, после чего мы делим его на размер окна,
            # чтобы результаты для разных размеров были сравнимы
            rmse.append(self.scorer.score(model.predict(first), second.tags_distribution) / first.seconds)

        # У полученных результатов считаем медиану и среднее
        return Metrics(statistics.median(rmse), statistics.mean(rmse))

In [4]:
# Функция тестирования
def test(posts: List[Post]):
    window_manager = WindowsManager(posts)
    # Получаем список размеров окон, на которых будем проводить тесты
    sizes = list(window_manager.windows_sizes_range(min_window_size, tests_number, min_windows_number))
    for size in sizes:
        # Получили окна заданного размера
        windows = window_manager.windows(size)
        # Делим окна на train и test
        train, test = np.split(windows, [int(len(windows) * train_share)])
        # Проводим тест для всех заданных моделей
        for model_test in tests:
            model = model_test.factory()
            model.fit(train)
            metrics_calculator = MetricsCalculator(test)
            model_test.score.append(metrics_calculator.metrics(model).avg_rmse)

    plot = PlotBuilder("", "window size (sec.)", "RMSE")
    for model_test in tests:
        plot.add_trace(sizes, model_test.score, model_test.name)
    plot.plot_offline()

In [5]:
# Модели, участвующие в тесте
# Параметры моделей:
# TagModel(<кол-во окон, на которых считается статистика>, <применяемая модель>)
tests = [
    ModelTest("zero", lambda: ZeroModel()),
    ModelTest("baseline", lambda: Baseline()),
    ModelTest("SGDRegressor", lambda: TagModel(3, SGDRegressor(random_state=10))),
    ModelTest("MLPRegressor", lambda: TagModel(3, MLPRegressor(random_state=10))),
    ModelTest("Ansamble", lambda: Ensemble(5, 9, lambda: TagModel(1, SVR(), False))),
    #ModelTest("PassiveAggressiveRegressor",
    #          lambda: TagModel(3, PassiveAggressiveRegressor(random_state=10))),
    #ModelTest("SGDRegressor tagwise", lambda: TagwiseModel(3, lambda: SGDRegressor(random_state=10))),
    #ModelTest("MLPRegressor tagwise", lambda: TagwiseModel(3, lambda: MLPRegressor(random_state=10))),
    #ModelTest("PassiveAggressiveRegressor tagwise",
    #          lambda: TagwiseModel(3, lambda: PassiveAggressiveRegressor(random_state=10))),
]

# Сколько раз перезапускать тест, варьируя размер окна
tests_number = 100
# Минимальный размер окна в секундах
min_window_size = 200
# Минимальное количество окон, которое может участвовать в тесте
min_windows_number = 45

# Доля окон, отданная только на обучение
train_share = 0.2

# Как видоизменять теги
# tag_map = to_first_letter
# tag_map = to_numbered_group(5) # параметр -- число групп для разбиения
tag_map = to_same_tag

In [6]:
# Чтобы применить новые параметры, нужно перезапустить эту ячейку
posts = [Post.parse(text, time, tag_map) for text, time in raw_posts]
tags = set([tag for post in posts for tag in post.tags])
print("Tags total", len(tags))
test(posts)

Tags total 2021
